# NEM Open Data Extraction Tool

**The Objective**: to create a tool that can easily extract open data from nemweb.com.au and assemble it into an analysis-ready format.

* For the intitial testing phase, extracted data will be stored as CSV files
* Once the data extraction pipeline is set up, we can experiment with SQL databases

In [1]:
%load_ext autoreload
%autoreload 2

# Standard Python
import os
import pandas as pd

# My modules
from nem_tracker import NEM_tracker
from nem_extract import NEM_extractor
from nem_loader import NEM_loader

# Configuration
import config
config.use('config.json')

Value for DATA_PATH has been set!


### NEM Tracker

* `.bulk_update()` method will update all existing resources
* to add a new resource use `.update_resource(resource, new=True)`
* tracking CSVs in `.tracker_dir` keep track of resource URLs 

In [2]:
data_dir = os.getenv('DATA_PATH')
nem_trk = NEM_tracker(data_dir)
for k, v in nem_trk.resources.items():
    print(k, v['last_update'])

/Reports/Current/Operational_Demand/ACTUAL_DAILY/ 2020-04-12-12:55:10
/Reports/Current/Dispatch_SCADA/ 2020-04-12-12:55:10


In [3]:
resources_new = ['/Reports/Current/Next_Day_Intermittent_DS/']
nem_trk.add_resources(resources_new)
#nem_trk.bulk_update()

### NEM Extractor

In [4]:
nem_get = NEM_extractor(data_dir)
nem_get.select_resource()

[0] /Reports/Current/Operational_Demand/ACTUAL_DAILY/
[1] /Reports/Current/Dispatch_SCADA/
[2] /Reports/Current/Next_Day_Intermittent_DS/

Selected: /Reports/Current/Next_Day_Intermittent_DS/


In [5]:
nem_get.load_tracker_df()
nem_get.current_tracker_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
TIMESTAMP        60 non-null datetime64[ns]
VERSION          60 non-null object
DOWNLOADED       60 non-null bool
DOWNLOAD_DATE    60 non-null datetime64[ns]
URL              60 non-null object
dtypes: bool(1), datetime64[ns](2), object(2)
memory usage: 2.1+ KB


In [6]:
nem_get.set_download_df(5)
nem_get.download_df

,TIMESTAMP,VERSION,DOWNLOADED,DOWNLOAD_DATE,URL
55,2020-04-07,V0000000321370873,False,1900-01-01,http://nemweb.com.au/Reports/Current/Next_Day_...
56,2020-04-08,V0000000321415237,False,1900-01-01,http://nemweb.com.au/Reports/Current/Next_Day_...
57,2020-04-09,V0000000321457595,False,1900-01-01,http://nemweb.com.au/Reports/Current/Next_Day_...
58,2020-04-10,V0000000321498882,False,1900-01-01,http://nemweb.com.au/Reports/Current/Next_Day_...
59,2020-04-11,V0000000321540365,False,1900-01-01,http://nemweb.com.au/Reports/Current/Next_Day_...


In [7]:
nem_get.download_files()
nem_get.current_tracker_df.tail(7)

,TIMESTAMP,VERSION,DOWNLOADED,DOWNLOAD_DATE,URL
53,2020-04-05,V0000000321287023,False,1900-01-01 00:00:00,http://nemweb.com.au/Reports/Current/Next_Day_...
54,2020-04-06,V0000000321329144,False,1900-01-01 00:00:00,http://nemweb.com.au/Reports/Current/Next_Day_...
55,2020-04-07,V0000000321370873,True,2020-04-12 13:32:16,http://nemweb.com.au/Reports/Current/Next_Day_...
56,2020-04-08,V0000000321415237,True,2020-04-12 13:32:27,http://nemweb.com.au/Reports/Current/Next_Day_...
57,2020-04-09,V0000000321457595,True,2020-04-12 13:32:32,http://nemweb.com.au/Reports/Current/Next_Day_...
58,2020-04-10,V0000000321498882,True,2020-04-12 13:32:51,http://nemweb.com.au/Reports/Current/Next_Day_...
59,2020-04-11,V0000000321540365,True,2020-04-12 13:33:06,http://nemweb.com.au/Reports/Current/Next_Day_...


### Setting up the NEM Loader

* The tracker files should be useful as metadata for what's in the resource folders
  * Basically, if `DOWNLOADED==True`, the files should be found in the relevant resource folders
    * However, we may want to have file names tracked, or just use the last part of the `URL` field, which ideally would correspond to the file names

* Parsing the downloaded data files:
  * The `C` code appears to be used to signal the start/end of data files
  * `I` seems to be column names
  * `D` are the data rows

In [21]:
nem_load = NEM_loader(data_dir)
nem_load.get_available_files()

[0] /Reports/Current/Operational_Demand/ACTUAL_DAILY/
[1] /Reports/Current/Dispatch_SCADA/
[2] /Reports/Current/Next_Day_Intermittent_DS/

Selected: /Reports/Current/Next_Day_Intermittent_DS/


In [23]:
nem_load.set_read_list()
nem_load.files_to_read

['PUBLIC_NEXT_DAY_INTERMITTENT_DS_20200411_0000000321540365.CSV']

In [24]:
test = nem_load.process_read_list()
test.keys()

dict_keys(['PUBLIC_NEXT_DAY_INTERMITTENT_DS_20200411_0000000321540365.CSV'])

In [25]:
k = list(test.keys())[0]
test[k][0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263066 entries, 0 to 263065
Data columns (total 13 columns):
DEMAND                    263066 non-null object
INTERMITTENT_DS_RUN       263066 non-null object
1                         263066 non-null int64
RUN_DATETIME              263066 non-null object
DUID                      263066 non-null object
ORIGIN                    263066 non-null object
FORECAST_PRIORITY         263066 non-null int64
OFFERDATETIME             263066 non-null object
PARTICIPANT_TIMESTAMP     242078 non-null object
SUPPRESSED_PARTICIPANT    261338 non-null float64
SUPPRESSED_AEMO           261338 non-null float64
TRANSACTION_ID            214106 non-null object
LASTCHANGED               263066 non-null object
dtypes: float64(2), int64(2), object(9)
memory usage: 26.1+ MB


In [29]:
test[k][1].sample(7)

,DEMAND,INTERMITTENT_DS_PRED,1,RUN_DATETIME,DUID,ORIGIN,FORECAST_PRIORITY,OFFERDATETIME,INTERVAL_DATETIME,FORECAST_MEAN,FORECAST_POE10,FORECAST_POE50,FORECAST_POE90
40693,DEMAND,INTERMITTENT_DS_PRED,1,2020/04/11 08:55:00,KSP1,KIDSTON,3,2020/04/11 08:46:25,2020/04/11 08:55:00,NaN,NaN,0.0000,NaN
172953,DEMAND,INTERMITTENT_DS_PRED,1,2020/04/11 08:50:00,KSP1,KIDSTON,98,2020/04/11 08:42:27,2020/04/11 08:50:00,NaN,NaN,0.0000,NaN
12296,DEMAND,INTERMITTENT_DS_PRED,1,2020/04/11 13:10:00,CROWLWF1,CROWLDWF,1,2020/04/11 13:00:08,2020/04/11 13:10:00,NaN,NaN,0.0000,NaN
194451,DEMAND,INTERMITTENT_DS_PRED,1,2020/04/11 07:25:00,LILYSF1,EEQLILY,1,2020/04/11 07:17:14,2020/04/11 07:25:00,NaN,NaN,48.1860,NaN
211388,DEMAND,INTERMITTENT_DS_PRED,1,2020/04/11 08:50:00,HAUGHT11,HAUGHTSF,98,2020/04/11 08:42:56,2020/04/11 08:50:00,NaN,NaN,0.0000,NaN
96475,DEMAND,INTERMITTENT_DS_PRED,1,2020/04/11 11:05:00,KSP1,KIDSTON,101,2020/04/11 10:58:08,2020/04/11 11:05:00,NaN,NaN,45.7974,NaN
172062,DEMAND,INTERMITTENT_DS_PRED,1,2020/04/12 02:30:00,TARALGA1,TARALGA,2,2020/04/12 02:21:22,2020/04/12 02:30:00,NaN,NaN,0.0000,NaN
